In [ ]:
# -*- coding:utf-8 -*-

import cv2
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff


def FouriesTransformAndFilterBackToImg(img):
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    #magnitude_spectrum = 20 * np.log(np.abs(fshift))

    rows, cols = img.shape
    crow,ccol = rows/2 , cols/2
    fshift[crow-500:crow+500, ccol-2500:ccol+2500] = 0
    #gsift = np.zeros(fshift.shape,np.complex64)
    #gsift[crow-200:crow+200, ccol-1000:ccol+1000] = fshift[crow-200:crow+200, ccol-1000:ccol+1000]

    f_ishift = np.fft.ifftshift(fshift)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.abs(img_back)

    return img_back


if __name__ == '__main__':

    print('Handling Four Band.')

    file_2015 = './20170905_preliminary/preliminary/quickbird2015.tif'
    file_2017 = './20170905_preliminary/preliminary/quickbird2017.tif'

    for band in range(0,2):

        print('Handling Band ' + str(band))

        sat_2015 = tiff.imread(file_2015).transpose([1, 2, 0])
        img_outred_2015 = sat_2015[:,:,band]
        img_outred_2015 = (img_outred_2015 - np.mean(img_outred_2015)) / np.std(img_outred_2015)
        img_outred_2015 = img_outred_2015 * 255
        sat_2015 = None

        img_2015_pass = FouriesTransformAndFilterBackToImg(img_outred_2015)

        #tiff.imsave('./Out/img_2015_pass_'+str(band)+'.tiff', img_2015_pass)

        img_outred_2015 = None


        sat_2017 = tiff.imread(file_2017).transpose([1, 2, 0])
        img_outred_2017 = sat_2017[:,:,band]
        img_outred_2017 = (img_outred_2017 - np.mean(img_outred_2017)) / np.std(img_outred_2017)
        img_outred_2017 = img_outred_2017 * 255
        sat_2017 = None

        img_2017_pass = FouriesTransformAndFilterBackToImg(img_outred_2017)

        #tiff.imsave('./Out/img_2017_pass_'+str(band)+'.tiff', img_2017_pass)
        img_outred_2017 = None

        diff_one = img_2017_pass - img_2015_pass

        img_2015_pass = None
        img_2017_pass = None

        # map to min max 255
        min_out_a = np.min(diff_one)
        max_out_a = np.max(diff_one)
        out_a = ( diff_one - min_out_a ) * 255.0 / max_out_a
        diff_one = None


        ## 高斯模糊啊
        #blur = diff_one
        #blur = cv2.GaussianBlur(out_a,(15,15),1)

        #thread_mean = np.mean(blur)  # 均值，改为中位数，试试
        #thread_sigma = np.std(blur)
        #thread_median = np.median(blur)

        #print('mean is ' + str(thread_mean))
        #print('median is ' + str(thread_median))

        #thread = thread_mean + 1 * thread_sigma
        #tf = blur >= thread
        #out_b = tf.astype(np.uint8)
        tiff.imsave('./Out/band_' + str(band) + 'pass.tiff', out_a)


"""
out_a_2 = np.square(out_a)
out_a_2 = np.sqrt(out_a_2)
thread_mean = np.mean(out_a_2)  # 均值，改为中位数，试试
thread_sigma = np.std(out_a_2)
thread_median = np.median(out_a_2)

print('mean is ' + str(thread_mean))
print('mean is ' + str(thread_median))

thread = thread_mean + 3 * thread_sigma
tf = out_a_2 >= thread
out_b = tf.astype(np.uint8)
tiff.imsave('./Match/Out/sub920_mean_3sigma.tiff', out_b)
"""
